# ChIP AML PiPeline v2

In [249]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../..')
import itertools
from scipy import stats
import numpy as np

from JKBio.epigenetics import ChIP_helper as chiphelper
from JKBio import Helper as helper
import igv
import SimpSOM as sps
from scipy import stats

import seaborn as sns
from matplotlib import cm
from matplotlib import pyplot as plt
from IPython.display import IFrame
import seaborn as sns
from bokeh.plotting import *
import igv

import numba
from numba import jit

from scipy.cluster.hierarchy import linkage, leaves_list
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from IPython.display import IFrame

from pybedtools import BedTool
import pyBigWig

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
project="Cobinding_ChIP"
version="v2"
merging_version="remove_single"
window="150"

In [394]:
%store -r merged
%store -r promoters
#ATAC
%store -r a 
%store -r chrombed
%store -r mergedpeak
%store -r endata
%store -r l

Stored 'merged' (DataFrame)
Stored 'a' (list)
Stored 'chrombed' (DataFrame)
Stored 'endata' (int)


UsageError: Unknown variable 'l'


## Doing Motif analysis

In [ ]:
! mkdir ../../data/MEME

In [ ]:
## computing motif across the open region of the genome of MV411 from ATACseq with MEME.mast
! bedtools getfasta -fi ../../data/ref/Homo_sapiens_assembly38.fasta -bed ../data/$project/ATACseq/ATAC_MV411.mRp.clN_peaks.broadPeak | fold -w 500 > ../../data/MEME/ATAC.fasta
! mkdir ../../data/MEME/ATAC/
#! export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && mast --oc ../../data/MEME/ATAC/ --remcorr ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/MEME/ATAC.fasta
#! export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && mcast --oc ../../data/MEME/ATAC/ --max-gap 80 ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/MEME/ATAC.fasta
#! export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && fimo --oc ../../data/MEME/ATAC/ ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/MEME/ATAC.fasta
# ! gff2bed < ../../data/MEME/ATAC/fimo.gff > ../../data/MEME/ATAC/fimo.gff.bed

In [ ]:
merged['strand'] = '.'
merged[merged.columns[[0,1,2,3,4,-1]]].to_csv('../results/'+project+"/"+version+'_'+merging_version+'_'+window+'_ merged_true.bed',index=False, header=False sep='\t')

In [ ]:
## What are the motifs enriched for each cluster groups in the conscensus peak set? 
! bedtools getfasta -fi ../../data/ref/Homo_sapiens_assembly38.fasta -bed ../results/$project/$version_$merging_version_$window_merged_true.bed | fold -w 500 > ../../data/MEME/merged$version_$merging_version_$window.fasta
#! mkdir ../../data/MEME/merged/
! export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && mast --oc ../../data/MEME/merged/ --remcorr ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/MEME/merged.fasta
! export PATH=$HOME/meme/bin:$HOME/meme/libexec/meme-5.1.1:$PATH && fimo --oc ../../data/MEME/merged/ ../../motif_databases/HUMAN/HOCOMOCOv11_full_HUMAN_mono_meme_format.meme ../../data/MEME/merged.fasta
! gff2bed < ../../data/MEME/merged/fimo.gff > ../../data/MEME/merged/fimo.gff.bed

In [ ]:
cols = ['ZEB2',
 'MYBL2',
 'ZMYND8',
 'IKZF1',
 'SP1',
 'RUNX1',
 'PU1_FLAG',
 'LDB1',
 'IRF8',
 'JUND',
 'MEF2C',
 'BRD4',
 'CDK8',
 'CDK13',
 'GATA2',
 'GFI1',
 'CEBPA',
 'CEBPB',
 'KMT2A',
 'AFF4',
 "AFF1",
 'MYC',
 'LMO2',
 'RUNX2',
 'FOXP1',
 'MYB',
 'ETV6',
 'MAX',
 'MEIS1',
 'FLI1',
 'LYL1',
 'MEF2D_FLAG',
 'ELF2']
rename = {'MEF2D_FLAG':'MEF2D','MYBL2':'MYBB',"PU1":'SPI1','PU1_FLAG':'SPI1'}
# TODO: find ZEB2, ZMYND8, LMO2, LDB1, CDK13 motifs

for k, val in rename.items():
     if k in cols:
        cols.remove(k)
        cols.append(val)

In [ ]:
## What are the motifs of our CRC members in ATACseq but not in our matrix
merged_motif = pd.read_csv('../../data/MEME/merged/fimo.gff.bed', sep='\t',skiprows=0,index_col=None, names=['pos',"fimo", "nucleotide_motif","relStart","relEnd","pval","strand",".","data"])

merged_motif['tf']=[i[5:].split('_HUMAN')[0] for i in merged_motif.data]
merged_motif = merged_motif[merged_motif.tf.isin(cols)]
merged_motif['chr'] = [i.split(':')[0][3:] for i in merged_motif.index]
merged_motif['start'] = [i.split(':')[1].split('-')[0] for i in merged_motif.index]
merged_motif['end'] = [i.split(':')[1].split('-')[1] for i in merged_motif.index]

In [ ]:
merged_motif['end'] = [i.split(':')[1].split('-')[1] for i in merged_motif.index]

In [ ]:
merged_motif

In [ ]:
## what enrichment do we have in each group? what enrichment do we have for each ChipSeq?

In [ ]:
## computing predicted motif for each TF from Chip data MEME-Chip

In [ ]:
## comparing with the litterature using MEME.tomtom

In [ ]:
### loading motifs from 

## Assigning genes

### based on closest expressed gene

In [ ]:
chiphelper.AssignToClosestExpressed()

In [ ]:
### recompute cobinding based on this.

In [ ]:
## redo the plots. do we get better plots?/correlations?...

### based on the ABC model

![](images/ABCtitle.png)

They tested a new model based on and validated by CRISPRi-FlowFISH which is basically able to find enhancer mapping to genes. 
They used it to compute their model's Accuracy and found a 70% accuracy compared to less than 50% for closest expressed gene. 

Way to integrate our HiC data (need ATAC-seq like data as well, but openly available) 


![](images/ABCmodel.png)

In [ ]:
Helper.scatter(TSNE(2,5).fit_transform(data.T), labels=zones.columns[11:],colors=labels)

In [ ]:
### recompute cobinding based on this.

In [ ]:
### redo the plots. do we get better looking plots?

In [ ]:
### compare presence of CTCF and transcription of linked RNA

## Predict

In [ ]:
## enrichment at the gene set level

In [ ]:
## are our cobinding clusters enriched in some sets of genes / CRC? 

In [ ]:
## make a linear model to with marks and cobinding data + motifs and TFs+cobinding motifs + expression of cobound proteins + expression , can we predict expression/dependenccy?

In [ ]:
## can we predict better for some set of genes / CRCs?

In [ ]:
## do the same on closest expressed gene

In [ ]:
## do this prediction on each enhancer.what is the best predicting enhancer? do that correlate with ABC model data?

In [ ]:
## do we get, for some gene, better single enhancer prediction?

In [ ]:
## what are the gene sets enriched in each clusters?? (based on TF cobinding or based on TF cobinding + )

## RNP data

In [ ]:
## can we predict RNP data (setting this TF to zero and lookingg at expected RNA change)
## do we see genes with opposite effects? 

In [ ]:
## if we add RNP data, can we increase our model's prediction? (we have expression change and we set all RNPed-TF values to 0) 

In [ ]:
### to predict remaining X% RNA expression
### to predict remaining X TFs RNP
### to predict regular RNA expression

In [ ]:
## same thing with filtering base on motif presence (actual DNA binding)
## same thing with closest expressed gene

In [ ]:
## Do on all cell lines

In [ ]:
## repeat the process across all samples with H3K27ac+RNAseq data we have.

In [ ]:
## call mutations from H3k27ac data

In [ ]:
## MEME analysis of likely bound TFs, except if TF is not expressed

In [ ]:
## compute enrichments

In [ ]:
## from same gene assignements, as found on the general consensus peak set, can we find good dependency/expression prediction

### Compare data with other labs (H3K27, HiC..)

we need to redo everything for similar normal cell type, getting TFs based on the CRC (find it with CRCmapper or on litterature)